In [6]:

import anndata as ad
import networkx as nx
import scanpy as sc
import scglue
from itertools import chain
from matplotlib import rcParams
import pandas as pd

import os
print(os.getcwd())
# os.chdir('/home/parallels/Downloads')
#
rna = ad.read_h5ad("Chen-2019-RNA.h5ad")
atac = ad.read_h5ad("Chen-2019-ATAC.h5ad")
rna.layers["counts"] = rna.X.copy()
sc.pp.highly_variable_genes(rna, n_top_genes=800, flavor="seurat_v3")
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
sc.pp.scale(rna)
sc.tl.pca(rna, n_comps=100, svd_solver="auto")
sc.pp.neighbors(rna, metric="cosine")
sc.tl.umap(rna)
scglue.data.lsi(atac, n_components=100, n_iter=15)


#
sc.pp.subsample(rna, n_obs=1000, random_state=42)
atac=atac[pd.Series(atac.obs_names).sample(1000),pd.Series(atac.var_names).sample(1000)].copy()

print(atac,rna)
#
scglue.data.get_gene_annotation(
    rna, gtf="gencode.vM25.chr_patch_hapl_scaff.annotation.gtf.gz",
    gtf_by="gene_name"
)
#
split = atac.var_names.str.split(r"[:-]")
atac.var["chrom"] = split.map(lambda x: x[0])
atac.var["chromStart"] = split.map(lambda x: x[1]).astype(int)
atac.var["chromEnd"] = split.map(lambda x: x[2]).astype(int)

guidance = scglue.genomics.rna_anchored_guidance_graph(rna, atac)# change name to prior
#scglue.graph.check_graph(guidance, [rna, atac])

# rna.write("rna-pp-seq.h5ad", compression="gzip")
# atac.write("atac-pp-seq.h5ad", compression="gzip")
# nx.write_graphml(guidance, "guidance-seq.graphml.gz")

/Users/meiqiliu/PycharmProjects/GLUE3/evaluation/workflow/scripts
AnnData object with n_obs × n_vars = 1000 × 1000
    obs: 'domain', 'cell_type'
    obsm: 'X_lsi' AnnData object with n_obs × n_vars = 1000 × 28930
    obs: 'domain', 'cell_type'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'hvg', 'log1p', 'pca', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'distances', 'connectivities'


window_graph:   0%|          | 0/28930 [00:00<?, ?it/s]

In [8]:
print(rna)

AnnData object with n_obs × n_vars = 1000 × 28930
    obs: 'domain', 'cell_type'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_type', 'mgi_id', 'havana_gene', 'tag'
    uns: 'hvg', 'log1p', 'pca', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'distances', 'connectivities'


In [9]:
atac

AnnData object with n_obs × n_vars = 1000 × 1000
    obs: 'domain', 'cell_type'
    var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
    obsm: 'X_lsi'

In [10]:
rna.write("rna-pp-seq.h5ad", compression="gzip")
atac.write("atac-pp-seq.h5ad", compression="gzip")
nx.write_graphml(guidance, "guidance-seq.graphml.gz")

In [11]:
import bindome
mypath='mm10.fa'
seq2=bindome.tl.get_sequences_from_bed(atac.var[['chrom', 'chromStart', 'chromEnd']], genome='mm10', uppercase=True,gen_path=mypath)
seq2

/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/tmp1qnu8lla
genome mm10 True
mm10.fa
True mm10.fa
running bedtools...
bedtools getfasta -fi mm10.fa -bed /var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/tmp1qnu8lla -fo /var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/tmp3yzd1zi0


[['chr16:39675797-39676138',
  'ATACCTAACTAAATTATGTTGCTATGATTTATTTGTCTCTTAGCTCCTTTATACCATTCTAAATTGTTATGTTATCACCACTTCCTTCTCTGACAACTTCTGTAATCCACATATGCTGTAATTTAACTCACATATGTTCTCACTTGCTGCCTATACATTGAATCCATTCCATCCACGCTCCATAAATCCTCATCAGCGAACACAGTGATCAACAGCAGAGATTCCTATGCCTGGCCCATCTGGCTTTCCAGACAGTTTTCCAGCCTAGATTTGCAGTTATTTGTCTACTGAAATTTCTATTCCATCTATTTAAAAAAAAAGAAAGAAAGAAAGAAAGAAAA'],
 ['chrX:140511039-140511487',
  'TGGAAAAATCTTAGGCACTTCAGTCTTCCTTGCTGCCTTTCTTCCCTAACCCAGTCAGTCATCCAGAACTATCCAGGAGCCGGCAGGCTGGGCGGGCCACCCAGTACCCAGGCCCGCCCTGTCTGGCGGCCGCATGGAGAGCCGCATGCCTTCTGCCCCTGGCTGGCCACTGAACCGCGTGGGGAGCCATGTGCTGCCGGTCACCAGGGAGGGGCAGCCCAACAGGATCGCCGGCTTGGATGCCACCAAGAACCAACCGCAGAGCCTTTTCAGAGGCCCGAGGGAGGGGAAGAAAGTGGGCAGGACAGGAAAGGAGAGCTTGCAGCTAGGATGCCTGGGTAATTTATTGTTGCGTGAGAGTGTCTCAGAGCAAGCAGCCCTGATTGAGGTCCAGACATGGGCAAGTCATTCCAAGGGAAAAGAGGTTATAACCACAAAAAGAGAACAT'],
 ['chr17:9683613-9684123',
  'TCTTTGTTACATCTTCATCATCTCTTAGACTGGGATTTCTACAGTAATTATGAAGATGGTTGTCCCTGTGTCCAGGCTGTGGCCATACTGAATTCCTGGAGTAATCCC

In [12]:
len(seq2)

1000

In [13]:
atac

AnnData object with n_obs × n_vars = 1000 × 1000
    obs: 'domain', 'cell_type'
    var: 'chrom', 'chromStart', 'chromEnd', 'highly_variable'
    obsm: 'X_lsi'

In [16]:
rna

AnnData object with n_obs × n_vars = 1000 × 28930
    obs: 'domain', 'cell_type'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'gene_id', 'gene_type', 'mgi_id', 'havana_gene', 'tag'
    uns: 'hvg', 'log1p', 'pca', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'distances', 'connectivities'

In [22]:
a1=torch.rand(2,4,5)
a2=torch.rand(10,4)
print(a2.sum(dim=1, keepdim=True).shape)# l is sum of each row 
a1.sum(dim=1, keepdim=True).shape# for 3d input
print(a1)
a1.sum(dim=1, keepdim=True)

torch.Size([10, 1])
tensor([[[0.6518, 0.5426, 0.8739, 0.1767, 0.9127],
         [0.3172, 0.3359, 0.1162, 0.7758, 0.1839],
         [0.2336, 0.6298, 0.8169, 0.9650, 0.7841],
         [0.4879, 0.5398, 0.1265, 0.3375, 0.6845]],

        [[0.6273, 0.7716, 0.3011, 0.7308, 0.9428],
         [0.0426, 0.4170, 0.7855, 0.1332, 0.8302],
         [0.0965, 0.9850, 0.6774, 0.1111, 0.5012],
         [0.6785, 0.3419, 0.5334, 0.5987, 0.1948]]])


tensor([[[1.6905, 2.0480, 1.9336, 2.2550, 2.5652]],

        [[1.4450, 2.5155, 2.2974, 1.5737, 2.4690]]])

In [23]:
print(0.6518+0.3172+0.2336+0.4879)

1.6905000000000001


In [27]:
(a2 * (1e4 / a2.sum(dim=1, keepdim=True))).log1p()#shape 10,4

tensor([[8.1252, 8.0628, 7.7572, 7.0148],
        [8.4446, 7.1034, 8.2261, 5.9938],
        [7.9734, 7.7094, 7.6461, 7.9301],
        [6.6205, 8.4789, 6.2324, 8.2770],
        [7.2777, 7.5373, 7.7727, 8.3675],
        [8.1640, 7.5150, 6.2823, 8.3239],
        [8.3199, 3.8153, 8.4828, 6.9303],
        [8.4259, 7.5829, 3.3044, 8.1458],
        [8.0743, 7.0066, 8.2260, 7.5766],
        [8.6992, 6.1928, 8.1029, 5.3580]])

In [31]:
(a1 * (1e4 / a1.sum(dim=1, keepdim=True))).log1p()

tensor([[[8.2576, 7.8825, 8.4165, 6.6649, 8.1772],
         [7.5377, 7.4030, 6.4003, 8.1437, 6.5765],
         [7.2319, 8.0314, 8.3489, 8.3618, 8.0254],
         [7.9680, 7.8772, 6.4852, 7.3117, 7.8896]],

        [[8.3762, 8.0289, 7.1790, 8.4435, 8.2479],
         [5.6901, 7.4138, 8.1374, 6.7419, 8.1207],
         [6.5060, 8.2730, 7.9894, 6.5606, 7.6164],
         [8.4546, 7.2154, 7.7505, 8.2442, 6.6720]]])

In [32]:
a1

tensor([[[0.6518, 0.5426, 0.8739, 0.1767, 0.9127],
         [0.3172, 0.3359, 0.1162, 0.7758, 0.1839],
         [0.2336, 0.6298, 0.8169, 0.9650, 0.7841],
         [0.4879, 0.5398, 0.1265, 0.3375, 0.6845]],

        [[0.6273, 0.7716, 0.3011, 0.7308, 0.9428],
         [0.0426, 0.4170, 0.7855, 0.1332, 0.8302],
         [0.0965, 0.9850, 0.6774, 0.1111, 0.5012],
         [0.6785, 0.3419, 0.5334, 0.5987, 0.1948]]])